In [11]:
import main as edgeml
import requests as req
import random
import pandas as pd
import tsfresh

In [12]:
key = "RuW7CkG0keGCn9OH7yG40XY3nZz4Vfdl+gUgvpeiqxOP1eIwCwg90BjMrQVQV5fz3/zK1zBZ4uP+WFelZUmYTw=="
startTime = 1618760115000
collector = edgeml.datasetCollector("http://localhost",
                                    key,
                                    "Example Dataset",
                                    False)

labels = [random.randint(1,3) for i in range(500)]
for i in range(500):
    collector.addDataPoint("Accelerometer", random.randint(1,50)/10.0, startTime + i)

collector.onComplete()

In [13]:
project = edgeml.getProject("http://localhost", key)
dataPoints = project['datasets'][0]['sensors'][0]['data']
print(dataPoints)

[{'timestamp': 1618760115000, 'datapoint': 1}, {'timestamp': 1618760115001, 'datapoint': 1.9}, {'timestamp': 1618760115002, 'datapoint': 4.8}, {'timestamp': 1618760115003, 'datapoint': 0.6}, {'timestamp': 1618760115004, 'datapoint': 4.3}, {'timestamp': 1618760115005, 'datapoint': 0.1}, {'timestamp': 1618760115006, 'datapoint': 4.8}, {'timestamp': 1618760115007, 'datapoint': 4.6}, {'timestamp': 1618760115008, 'datapoint': 4.7}, {'timestamp': 1618760115009, 'datapoint': 1.5}, {'timestamp': 1618760115010, 'datapoint': 3.4}, {'timestamp': 1618760115011, 'datapoint': 5}, {'timestamp': 1618760115012, 'datapoint': 2.7}, {'timestamp': 1618760115013, 'datapoint': 2.7}, {'timestamp': 1618760115014, 'datapoint': 2.2}, {'timestamp': 1618760115015, 'datapoint': 2.7}, {'timestamp': 1618760115016, 'datapoint': 4}, {'timestamp': 1618760115017, 'datapoint': 4.4}, {'timestamp': 1618760115018, 'datapoint': 4.6}, {'timestamp': 1618760115019, 'datapoint': 4.4}, {'timestamp': 1618760115020, 'datapoint': 1.3

In [14]:
values = {'id': [i for i in range(500)], 'sensor1': []}
for dataPoint in dataPoints:
    values['sensor1'].append(dataPoint['datapoint'])
dataFrame = pd.DataFrame(values)
print(dataFrame)

      id  sensor1
0      0      1.0
1      1      1.9
2      2      4.8
3      3      0.6
4      4      4.3
..   ...      ...
495  495      3.3
496  496      4.0
497  497      4.8
498  498      0.1
499  499      2.8

[500 rows x 2 columns]


In [15]:
settings = tsfresh.feature_extraction.settings.MinimalFCParameters()
extracted = tsfresh.extract_features(dataFrame, column_id="id", default_fc_parameters=settings)
print(extracted)

Feature Extraction: 100%|██████████| 30/30 [00:00<00:00, 327.22it/s]

     sensor1__sum_values  sensor1__median  sensor1__mean  sensor1__length  \
0                    1.0              1.0            1.0              1.0   
1                    1.9              1.9            1.9              1.0   
2                    4.8              4.8            4.8              1.0   
3                    0.6              0.6            0.6              1.0   
4                    4.3              4.3            4.3              1.0   
..                   ...              ...            ...              ...   
495                  3.3              3.3            3.3              1.0   
496                  4.0              4.0            4.0              1.0   
497                  4.8              4.8            4.8              1.0   
498                  0.1              0.1            0.1              1.0   
499                  2.8              2.8            2.8              1.0   

     sensor1__standard_deviation  sensor1__variance  \
0                   

In [16]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(extracted, labels, random_state = 5)


In [17]:
from sklearn.preprocessing import RobustScaler
scaler = RobustScaler()
scaler.fit(x_train)
trans_x_train = scaler.transform(x_train)
trans_x_test = scaler.transform(x_test)
trans_x_train = pd.DataFrame(trans_x_train,columns=x_train.columns)
trans_x_train.describe()

,sensor1__sum_values,sensor1__median,sensor1__mean,sensor1__length,sensor1__standard_deviation,sensor1__variance,sensor1__root_mean_square,sensor1__maximum,sensor1__minimum
count,375.000000,375.000000,375.000000,375.0,375.0,375.0,375.000000,375.000000,375.000000
mean,-0.008640,-0.008640,-0.008640,0.0,0.0,0.0,-0.008640,-0.008640,-0.008640
std,0.581517,0.581517,0.581517,0.0,0.0,0.0,0.581517,0.581517,0.581517
min,-1.000000,-1.000000,-1.000000,0.0,0.0,0.0,-1.000000,-1.000000,-1.000000
25%,-0.520000,-0.520000,-0.520000,0.0,0.0,0.0,-0.520000,-0.520000,-0.520000
50%,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.000000
75%,0.480000,0.480000,0.480000,0.0,0.0,0.0,0.480000,0.480000,0.480000
max,0.960000,0.960000,0.960000,0.0,0.0,0.0,0.960000,0.960000,0.960000


In [18]:
from sklearn.ensemble import RandomForestClassifier
clf = RandomForestClassifier()
clf.fit(trans_x_train,y_train)


RandomForestClassifier()

In [19]:
from sklearn.metrics import accuracy_score
prediction = clf.predict(trans_x_train)
print("accuracy_score train :", accuracy_score(y_train,prediction))

accuracy_score train : 0.5066666666666667
